# Retriever - Implantação

O objetivo deste componente é elencar a probabilidade de um conjunto de contextos conter a resposta a uma dada pergunta.

Este componente utiliza diferentes modelos de similaridade entre textos para a sua inferência.<br>

A tabela de dados de entrada deve possuir uma coluna de contextos, em que cada linha representa um contexto diferente, e uma coluna de perguntas em que cada linha representa uma pergunta a ser utilizada. Note que para cada pergunta serão utilizados todos os contextos fornecidos para realização da inferência, e portanto, podem haver bem mais contextos do que perguntas.<br>
### **Em caso de dúvidas, consulte os [tutoriais da PlatIAgro](https://platiagro.github.io/tutorials/).**

## Declaração de Classe para Predições em Tempo Real

A tarefa de implantação cria um serviço REST para predições em tempo-real.<br>
Para isso você deve criar uma classe `Model` que implementa o método `predict`.

In [ ]:
!python -m spacy download pt_core_news_lg --quiet

In [1]:
%%writefile Model.py

import joblib
import numpy as np
import pandas as pd
from retriever import Retriever, MODEL_MAPPING

class Model:
    def __init__(self):
        self.loaded = False

    def load(self):
        
        # Load artifacts
        artifacts = joblib.load("/tmp/data/retriever.joblib")
        self.model_parameters = artifacts["model_parameters"]
        self.inference_parameters = artifacts["inference_parameters"]
        
        # Initialize retriever
        self.sim_model = MODEL_MAPPING[self.inference_parameters['similarity_model']](**self.model_parameters)
        self.retriever = Retriever(similarity_model=self.sim_model, **self.model_parameters)
        
        # Set model loaded
        self.loaded = True
        print("Loaded model")
    
    def class_names(self):
        column_names = list(self.inference_parameters['output_columns'])
        return column_names

    def predict(self, X, feature_names, meta=None):
        if not self.loaded:
            self.load()
            
        # Convert to dataframe
        if feature_names != []:
            df = pd.DataFrame(X, columns = feature_names)
            df = df[self.inference_parameters['input_columns']]
        else:
            df = pd.DataFrame(X, columns = self.inference_parameters['input_columns'])
            
        # Get unique questions
        unique_questions = list(df[self.inference_parameters['question_column_name']].unique())

        # Initialize output df
        output_df = pd.DataFrame(columns=self.inference_parameters['output_columns'])

        # Iterate over each unique question
        for question in unique_questions:

            # Filter df per question
            question_df = df[df[self.inference_parameters['question_column_name']] == question]

            # Get Contexts
            contexts = list(question_df[self.inference_parameters['context_column_name']])

            # Get indices
            context_ids = list(question_df.index)

            # Retrieve contexts
            topn_ids, topn_scores = self.retriever.retrieve_top(contexts, question, topn=self.inference_parameters['topn'], context_ids=context_ids)

            # Retrieved df
            retrieved_df = df.loc[topn_ids]
            retrieved_df.at[topn_ids, self.inference_parameters['proba_column_name']] = topn_scores

            # Save to output
            output_df = pd.concat((output_df, retrieved_df)).reset_index(drop=True)
        
        return output_df.to_numpy()

Overwriting Model.py
